## Day 8: Treetop Tree House

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.

First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible from outside the grid when looking directly along a row or column.

The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:

```
30373
25512
65332
33549
35390
```

In [1]:
import pytest
from icecream import ic

In [2]:
def load_input(filename):
    with open(filename, 'r') as f_input:
        tree_map = [line.strip() for line in f_input.readlines()]
    return (tree_map, len(tree_map[0]), len(tree_map))

In [3]:
tree_map, width, height = load_input('sample.txt')
assert width == 5
assert height == 5
assert tree_map[0][0] == '3'
assert tree_map[4][4] == '0'
assert tree_map[3][2] == '5'  # Access to map using m[y][x]

Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the interior nine trees to consider:

In [4]:
def is_visible(m, width, height, x, y):
    assert 0 <= x < width
    assert 0 <= y < height
    high = m[y][x]
    if all(m[y][left] < high for left in range(0, x)):
        return True
    if all(m[y][right] < high for right in range(x+1, width)):
        return True
    if all(m[top][x] < high for top in range(0, y)):
        return True
    if all(m[bottom][x] < high for bottom in range(y+1, height)):
        return True
    return False

- The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)

- The top-middle 5 is visible from the top and right.

- The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.

- The left-middle 5 is visible, but only from the right.

- The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.

- The right-middle 3 is visible from the right.

- In the bottom row, the middle 5 is visible, but the 3 and 4 are not.

In [5]:
tree_map, width, height = load_input('sample.txt')

assert is_visible(tree_map, width, height, 1, 1) is True
assert is_visible(tree_map, width, height, 2, 1) is True
assert is_visible(tree_map, width, height, 3, 1) is False

assert is_visible(tree_map, width, height, 1, 2) is True
assert is_visible(tree_map, width, height, 2, 2) is False
assert is_visible(tree_map, width, height, 3, 2) is True

assert is_visible(tree_map, width, height, 1, 3) is False
assert is_visible(tree_map, width, height, 2, 3) is True
assert is_visible(tree_map, width, height, 3, 3) is False

With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

In [6]:
def solution_one(filename):
    tree_map, width, height = load_input(filename)
    borders = (width + height) * 2 - 4
    inners = sum(
        is_visible(tree_map, width, height, x, y)
        for x in range(1, width-1)
        for y in range(1, height-1)
    )
    return borders + inners
    

In [7]:
assert solution_one('sample.txt') == 21

**Consider your map; how many trees are visible from outside the grid?**

In [8]:
sol = solution_one('input.txt')
print(f"Solution part one: {sol}")

Solution part one: 1827


## Part Two

Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of trees.

To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)

The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large eaves to keep it dry, so they wouldn't be able to see higher than the tree house anyway.

In the example above, consider the middle 5 in the second row:

```
30373
25❺12
65332
33549
35390
```

- Looking up, its view is not blocked; it can see **1** tree (of height 3).

- Looking left, its view is blocked immediately; it can see only **1** tree (of height 5, right next to it).

- Looking right, its view is not blocked; it can see **2** trees.

- Looking down, its view is blocked eventually; it can see **2** trees (one of height 3, then the tree of height 5 that blocks its view).

In [9]:
def line_of_sight(tree_high, tree_line):
    result = 0
    for high in tree_line:
        result += 1
        if high >= tree_high:
            break
    return result

assert line_of_sight('3', []) == 0
assert line_of_sight('3', ['2']) == 1
assert line_of_sight('3', ['3']) == 1
assert line_of_sight('3', ['4']) == 1
assert line_of_sight('0', ['2', '1', '2', '3']) == 1
assert line_of_sight('1', ['2', '1', '2', '3']) == 1
assert line_of_sight('2', ['2', '1', '2', '3']) == 1
assert line_of_sight('3', ['2', '1', '2', '3']) == 4
assert line_of_sight('3', '') == 0
assert line_of_sight('3', '2') == 1
assert line_of_sight('3', '3') == 1
assert line_of_sight('3', '4') == 1
assert line_of_sight('0', '2123') == 1
assert line_of_sight('1', '2123') == 1
assert line_of_sight('2', '2123') == 1
assert line_of_sight('3', '2123') == 4

In [10]:
def score_left(tree_map, x, y):
    if x <= 0:
        return 0
    high = tree_map[y][x]
    tree_line = tree_map[y][x-1::-1]
    return line_of_sight(high, tree_line)


def score_right(tree_map, x, y):
    high = tree_map[y][x]
    tree_line = tree_map[y][x+1:]
    return line_of_sight(high, tree_line)


def score_top(tree_map, x, y):
    if y <= 0:
        return 0
    high = tree_map[y][x]
    tree_line = [line[x] for line in tree_map[y-1::-1]]
    return line_of_sight(high, tree_line)


def score_bottom(tree_map, x, y):
    high = tree_map[y][x]
    tree_line = [line[x] for line in tree_map[y+1:]]
    return line_of_sight(high, tree_line)


tree_map, *_ = load_input('sample.txt')
assert score_left(tree_map, 2, 1) == 1
assert score_right(tree_map, 2, 1) == 2
assert score_top(tree_map, 2, 1) == 1
assert score_bottom(tree_map, 2, 1) == 2

In [11]:
tree_map, *_ = load_input('sample.txt')
x = 0
y = 0
assert ic(score_left(tree_map, x, y)) == 0
assert ic(score_right(tree_map, x, y)) == 2
assert ic(score_top(tree_map, x, y)) == 0
assert ic(score_bottom(tree_map, x, y)) == 2

ic| score_left(tree_map, x, y): 0
ic| score_right(tree_map, x, y): 2
ic| score_top(tree_map, x, y): 0
ic| score_bottom(tree_map, x, y): 2


In [12]:
tree_map, *_ = load_input('sample.txt')
x = 0
y = 2
assert ic(score_left(tree_map, x, y)) == 0
assert ic(score_right(tree_map, x, y)) == 4
assert ic(score_top(tree_map, x, y)) == 2
assert ic(score_bottom(tree_map, x, y)) == 2

ic| score_left(tree_map, x, y): 0
ic| score_right(tree_map, x, y): 4
ic| score_top(tree_map, x, y): 2
ic| score_bottom(tree_map, x, y): 2


A tree's **scenic score** is found by multiplying together its viewing distance in each of the four directions. 

In [13]:
def scenic_score(tree_map, x, y):
    return (
        max(1, score_right(tree_map, x, y))
        * max(1, score_bottom(tree_map, x, y))
        * max(1, score_left(tree_map, x, y))
        * max(1, score_top(tree_map, x, y))
    )

For this tree, this is **4** (found by multiplying 

$$ 1 \times 1 \times 2 \times 2 $$

However, you can do even better: consider the tree of height 5 in the middle of the fourth row:

```
30373
25512
65332
33❺49
35390
```

- Looking up, its view is blocked at 2 trees (by another tree with a height of 5).
- Looking left, its view is not blocked; it can see 2 trees.
- Looking down, its view is also not blocked; it can see 1 tree.
- Looking right, its view is blocked at 2 trees (by a massive tree of height 9).

In [14]:
tree_map, *_ = load_input('sample.txt')

assert score_left(tree_map, 2, 3) == 2
assert score_right(tree_map, 2, 3) == 2
assert score_top(tree_map, 2, 3) == 2
assert score_bottom(tree_map, 2, 3) == 1

This tree's scenic score is **8**: 

$$ 2 \times 2 \times 1 \times 2 $$

In [15]:
tree_map, *_ = load_input('sample.txt')

assert scenic_score(tree_map, 2, 3) == 8

this is the ideal spot for the tree house.

In [16]:
def solution_two(filename):
    tree_map, width, height = load_input(filename)
    solutions = [
        (scenic_score(tree_map, x, y), x, y)
        for y in range(1, height - 1)
        for x in range(1, width - 1)
    ]
    solutions = sorted(solutions, reverse=True)
    return solutions[0]

solution_two('sample.txt')
assert ic(solution_two('sample.txt')) == (8, 2, 3)

ic| solution_two('sample.txt'): (8, 2, 3)


**Consider each tree on your map. What is the highest scenic score possible for any tree?**

In [17]:
sol, _, _ = solution_two('input.txt')
print(f"Solution part two: {sol}")

Solution part two: 335580
